# Portfolio aggregations
In this notebook we'll give some examples on how the portfolio aggregation methods can be used.

### Setting up
We'll set up the imports, data providers, and load the portfolio. For more details about this, please have a look at notebook one (getting started).

In [2]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_valuation_protocol import TargetValuationProtocol
%aimport -pandas
import pandas as pd

In [3]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")
companies = portfolio[["company_id", "company_name"]].to_dict(orient="records")

company_data = SBTi.data.get_company_data([provider], companies)
targets = SBTi.data.get_targets([provider], companies)
portfolio_data = pd.merge(left=company_data, right=targets, how='outer', on=['company_name', 'company_id'])

target_valuation_protocol = TargetValuationProtocol(portfolio_data, company_data)
portfolio_data = target_valuation_protocol.target_valuation_protocol()
portfolio_data = pd.merge(left=portfolio_data, right=portfolio, how='outer', on=['company_name', 'company_id'])

In [4]:
temperature_score = TemperatureScore()
amended_portfolio = temperature_score.calculate(portfolio_data, ["investment_value"])

### Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done use different aggregation methods. The termperature scores are calculated per time-frame/scope combination.

In [5]:
def print_aggregations(aggregations):
    for time_frame, time_frame_values in aggregations.items():
        for scope, scope_values in time_frame_values.items():
            print("{} - {}: {} degrees celcius".format(time_frame, scope, scope_values["all"]["score"]))

#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

In [6]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.WATS, []))

short - s1s2: 1.4629 degrees celcius
short - s3: 2.3409 degrees celcius
short - s1s2s3: 2.0977 degrees celcius
mid - s1s2: 2.5923 degrees celcius
mid - s3: 2.9923 degrees celcius
mid - s1s2s3: 2.8998 degrees celcius
long - s1s2: 3.0814 degrees celcius
long - s3: 3.191 degrees celcius
long - s1s2s3: 3.155 degrees celcius


#### TETS
Total emissions weighted temperature score (TETS): Temperature scores are allocated based on historical emission weights using total company emissions. 

In [7]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.TETS, []))

short - s1s2: 1.6343 degrees celcius
short - s3: 2.3879 degrees celcius
short - s1s2s3: 2.0774 degrees celcius
mid - s1s2: 2.681 degrees celcius
mid - s3: 2.3564 degrees celcius
mid - s1s2s3: 2.5294 degrees celcius
long - s1s2: 2.8646 degrees celcius
long - s3: 3.1986 degrees celcius
long - s1s2s3: 3.0562 degrees celcius


#### MOTS
Market Owned emissions weighted temperature score (MOTS): Temperature scores are allocated based on an equity ownership approach.

In [8]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.MOTS, []))

short - s1s2: 2.2805 degrees celcius
short - s3: 2.251 degrees celcius
short - s1s2s3: 2.2568 degrees celcius
mid - s1s2: 2.5231 degrees celcius
mid - s3: 2.2171 degrees celcius
mid - s1s2s3: 2.3563 degrees celcius
long - s1s2: 3.0193 degrees celcius
long - s3: 3.1991 degrees celcius
long - s1s2s3: 3.1226 degrees celcius


#### EOTS
Enterprise Owned emissions weighted temperature score (EOTS): Temperature scores are allocated based
on an enterprise ownership approach. 

In [9]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.EOTS, []))

short - s1s2: 2.4141 degrees celcius
short - s3: 2.1479 degrees celcius
short - s1s2s3: 2.2477 degrees celcius
mid - s1s2: 2.2584 degrees celcius
mid - s3: 1.9427 degrees celcius
mid - s1s2s3: 2.1051 degrees celcius
long - s1s2: 3.0438 degrees celcius
long - s3: 3.1988 degrees celcius
long - s1s2s3: 3.1328 degrees celcius


#### ECOTS
Enterprise Value + Cash emissions weighted temperature score (ECOTS): Temperature scores are allocated based on an enterprise value (EV) plus cash & equivalents ownership approach. 

In [10]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.ECOTS, []))

short - s1s2: 2.3732 degrees celcius
short - s3: 2.2215 degrees celcius
short - s1s2s3: 2.2732 degrees celcius
mid - s1s2: 2.3592 degrees celcius
mid - s3: 2.148 degrees celcius
mid - s1s2s3: 2.2691 degrees celcius
long - s1s2: 3.0128 degrees celcius
long - s3: 3.1987 degrees celcius
long - s1s2s3: 3.1196 degrees celcius


#### AOTS
Total Assets emissions weighted temperature score (AOTS): Temperature scores are allocated based on a total assets ownership approach. 

In [11]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.AOTS, []))

short - s1s2: 1.6998 degrees celcius
short - s3: 0.4567 degrees celcius
short - s1s2s3: 0.7972 degrees celcius
mid - s1s2: 1.886 degrees celcius
mid - s3: 2.8486 degrees celcius
mid - s1s2s3: 2.5815 degrees celcius
long - s1s2: 2.8956 degrees celcius
long - s3: 3.1962 degrees celcius
long - s1s2s3: 3.0681 degrees celcius
